<a href="https://colab.research.google.com/github/YOU-nari/Stable-Diffusion-with-diffusers-Personal-modification-V3-github-/blob/main/Stable_Diffusion_with_%F0%9F%A7%A8_diffusers_Personal_modification_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title GPUの確認
!nvidia-smi

↑GPUが動く事確認してね     
もし動いてない場合は ランタイムのタイプをGPUにしてね

↓diffusersが更新されたら変更してね
現時点最新
diffusion:1.5
diffuser:0.7.2
Waifu:1.3
trinart:v2

In [ ]:
#@title Stable Diffusion diffusersをロード

#!pip install diffusers==0.7.2 transformers scipy ftfy accelerater
!pip install transformers scipy ftfy
!pip install --upgrade diffusers[torch]
#!huggingface-cli login
#自分のトークン記載 diffuser0.4.0からは1度ログ・インすれば不要らしい
#YOUR_TOKEN=""

In [ ]:
#@title Hugging Faceにログインする
#colabだとこの方法じゃないと駄目らしい
from huggingface_hub import notebook_login
notebook_login()

↓SEEDのランダム化と、Stable Diffusionモデルの選択を追加
モデルを切り替える時は"ランタイムの接続を解除してから再実行

In [ ]:
#@title このセルは一回だけ実行で良い、色々DLしてきます
import torch
from torch import autocast
from diffusers.models import AutoencoderKL

#SEEDをランダム化したいので追加
import random

#Xformersの実装をしたいけどよくわからんな
#!sudo docker run -it --gpus=all --ipc=host -v /home:/home nvcr.io/nvidia/pytorch:22.08-py3 bash
#!pip install --pre torch
#!pip install xformers pytorch_lightning numpy
#!pip3 install triton
!git clone https://github.com/openai/triton.git
%cd triton/python/
!pip install -e .

!pip install pytorch_lightning
# Then 
# !pip install git+https://github.com/facebookresearch/xformers@51dd119#egg=xformers

# Followed by
#!cd PATH_TO_DIFFUSER_FOLDER
#!git checkout memory_efficient_attention
#  !pip install -e . 

#import math
#import os

#import pytorch_lightning as pl
#import torch
#import torch.nn as nn

#from pytorch_lightning import Trainer, seed_everything
#from pytorch_lightning.utilities import rank_zero_info
#from torch.nn import functional as F
#from torch.utils.data import DataLoader, Dataset, RandomSampler

#from xformers.factory.model_factory import xFormer, xFormerConfig

#とりあえずコンパイル済みこれ入れときゃ動くからいいか
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

#@markdown Stable Diffusionモデルを選択

#@markdown 1:Normal 2:Waife 3:Trinart 4:Trinart-Waife-50-50
model = 1 #@param {type:"number"}

if model == 1:
 from diffusers import StableDiffusionPipeline , EulerAncestralDiscreteScheduler

# pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=YOUR_TOKEN
# ).to("cuda")
 vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
 euler_scheduler = EulerAncestralDiscreteScheduler.from_config("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
 pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", scheduler=euler_scheduler, torch_dtype=torch.float16 ,revision="fp16", vae=vae,custom_pipeline="lpw_stable_diffusion",
 ).to("cuda")
#NSFW回避処理
 def dummy(images, **kwargs): return images, False
 pipe.safety_checker = dummy
 print("Model:Normal")
elif model == 2:
 from diffusers import StableDiffusionPipeline,LMSDiscreteScheduler #DDIMScheduler
# StableDiffusionパイプラインの準備 1.3からschedulerが変わったのでコメント前のはコメントアウト
 pipe = StableDiffusionPipeline.from_pretrained(
     "hakurei/waifu-diffusion",
     torch_dtype=torch.float32,
     #revision="fp16",
     #scheduler=DDIMScheduler
     scheduler=LMSDiscreteScheduler(
         beta_start=0.00085,
         beta_end=0.012,
         beta_schedule="scaled_linear",
         #clip_sample=False,
         #set_alpha_to_one=False,
         num_train_timesteps=1000
     ),
     use_auth_token=YOUR_TOKEN
 ).to("cuda")
 def dummy(images, **kwargs): return images, False
 pipe.safety_checker = dummy
 print("Model:Waife")
elif model == 3:
 from diffusers import StableDiffusionPipeline

# StableDiffusionパイプラインの準備
 pipe = StableDiffusionPipeline.from_pretrained(
     "naclbit/trinart_stable_diffusion_v2", 
     revision="diffusers-60k",
     use_auth_token=YOUR_TOKEN
 ).to("cuda")
 def dummy(images, **kwargs): return images, False
 pipe.safety_checker = dummy
 print("Model:Trinart")
elif model == 4:
 from diffusers import StableDiffusionPipeline

# StableDiffusionパイプラインの準備
 pipe = StableDiffusionPipeline.from_pretrained(
     "doohickey/trinart-waifu-diffusion-50-50", 
     use_auth_token=YOUR_TOKEN
).to("cuda")
 def dummy(images, **kwargs): return images, False
 pipe.safety_checker = dummy
 print("Model:Trinart-Waifu-50-50")
else:
 print("入力エラー:数値を見直してください。")

ここから下は好き勝手いじって遊ぶためのパラメーター

num_inference_stepsは1~200

回数が多い程綺麗になるが、極端に大きくしても変わらない

推奨値は50

guidance_scaleは1~20

数値が大きい程Prompt内容に近くなるが多様性が無くなる

推奨値は7〜8.5くらい、30超えると破綻する

batch_size 増やすと施行数が増える?

実行速度も上がるがVRAMを大量に消費

In [ ]:
#@title 変数の設定

#変数宣言 何故かstepsは+1される
num_inference_steps = 10     #@param {type:"number"}      # Number of denoising steps
guidance_scale = 7.5        #@param {type:"number"}       # Scale for classifier-free guidance
#batch_size = 2 #@param {type:"number"}
# 画像のサイズ 512*512が初期値
height = 512        #@param {type:"number"}              # default height of Stable Diffusion
width = 512             #@param {type:"number"}          # default width of Stable Diffusion
#batch_size = 1 #@param {type:"number"}

In [ ]:
#@title 描画指示

#入力文字 ここに好きな禁則文字をいれてください

prompt = "masterpiece, ultra-detaled, best quality, A woman in a luxury dress with long hair with a beautiful and perfect symmetrical face blonde loose perm by mucha" #@param {type:"string"}

#ネガティブ 除外したい要素を入れる

negative_prompt = "pablo picasso,monet,dali,van gogh," #@param {type:"string"}

In [ ]:
#@title 画像生成回数とSEEDの種別

N = 1 #@param {type:"number"}
seed = 1 #@param {type:"number"}
seed_fix = True #@param {'type':'boolean'}
for i in range(N):
# SEED値、ここをかえると 同じ入力文字でも別の画像がでます
# seedを固定する時はseed_fixにチェック
   if seed_fix == True:
    seed = seed
    generator = torch.Generator("cuda").manual_seed(seed)
   else:
    seed = random.randrange(0, 2147483647, 1)
    generator = torch.Generator("cuda").manual_seed(seed)
   with autocast("cuda"):
  #VRAM消費を抑えたい場合有効に↓
    pipe.enable_attention_slicing()
    pipe.enable_xformers_memory_efficient_attention()
    with torch.inference_mode():
    #images = pipe(prompt, height = height, width = width, guidance_scale = guidance_scale, num_inference_steps = num_inference_steps,negative_prompt = negative_prompt, generator = generator).images
     images = pipe.text2img(prompt, negative_prompt = negative_prompt, width = width,height = height,guidance_scale = guidance_scale, num_inference_steps = num_inference_steps,max_embeddings_multiples = 8,generator = generator).images
#どのSEED値で描画されたか確認用
     print("SEED =",seed)
     display(images[0])#.save(f'output{i}.png')

もっと詳しく知りたい人は
https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=zHkHsdtnry57
こっちのちゃんとしたcolabを見るんだ！！！！


↓Waifu単独動作

ここから動かせば良い。
特に2次元はWifuが圧倒的

In [ ]:
 #@title Waifu Diffusionのダウンロードと設定
 #!pip install diffusers==0.7.2 transformers scipy ftfy accelerate
!pip install transformers scipy ftfy
!pip install --upgrade diffusers[torch]

#Xformersの実装をしたいけどよくわからんな
# !sudo docker run -it --gpus=all --ipc=host -v /home:/home nvcr.io/nvidia/pytorch:22.08-py3 bash

# Then 
# !pip install git+https://github.com/facebookresearch/xformers@51dd119#egg=xformers

# Followed by
# !cd PATH_TO_DIFFUSER_FOLDER
# !git checkout memory_efficient_attention
# !pip install -e . 

!git clone https://github.com/openai/triton.git
%cd triton/python/
!pip install -e .

!pip install pytorch_lightning
#とりあえずコンパイル済みこれ入れときゃ動くからいいか
!pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

#ランダム化したいので追加
import random
import torch
#from diffusers.models import AutoencoderKL
from torch import autocast
#from diffusers import DiffusionPipeline
 from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler #DDIMScheduler
# StableDiffusionパイプラインの準備
# pipe = StableDiffusionPipeline.from_pretrained(
#  "hakurei/waifu-diffusion",
#  torch_dtype=torch.float32,
  #revision="fp16",#←最新バージョンで廃止されてる
  #scheduler=DDIMScheduler(
#vae = AutoencoderKL.from_pretrained("waifu-diffusion-v1-4/vae/kl-f8-anime.ckpt")←仮 1.4リリース時に確認

 pipe = DiffusionPipeline.from_pretrained(
    'hakurei/waifu-diffusion',   
    custom_pipeline="lpw_stable_diffusion",
    #revision="fp32",
    torch_dtype=torch.float32,#vae=vae,
  scheduler=LMSDiscreteScheduler(
         beta_start=0.00085,
         beta_end=0.012,
         beta_schedule="scaled_linear",
         #.clip_sample=False,
         #set_alpha_to_one=False,
         num_train_timesteps=1000
     ),
     #use_auth_token=YOUR_TOKEN
 ).to("cuda")
#NSFW回避処理
def dummy(images, **kwargs): return images, False
pipe.safety_checker = dummy
#VRAM低消費※少し実行速度が落ちる
#pipe.enable_attention_slicing()

GUIでなくCUIで動くように直した。
ループ組んで指定した回数実行し続けるのでこっちのほうが使い勝手よい。
ランタイム切断でも再起動後継続する。

num_inference_stepsは1~200

回数が多い程綺麗になるが、極端に大きくしても変わらない

推奨値は50

guidance_scaleは1~20

数値が大きい程Prompt内容に近くなるが多様性が無くなる

推奨値は7〜8.5くらい、30超えると破綻する

batch_size 増やすと施行数が増える?

実行速度も上がるがVRAMを大量に消費

基本的には512×512が1番良い結果が出る

guidance_scale 1~30 大きい程promptに近い画像になるが多様性は無くなる

In [ ]:
#@title 変数の設定

#変数 初期値 512*512 scale:6 step:50
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}
guidance_scale = 6 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
#batch_size = 1 #@param {type:"number"}
#num_samples = 1 これは [prompt] * num_samplesで入れ子に指定回数分出力

In [1]:
#@title 描画指示
prompt = "cute cat ear maid" #@param {type:"string"}

negative_prompt = "cat ear" #@param {type:"string"}

In [ ]:
#@title ループ回数
N = 1 #@param {type:"number"}
seed = 1 #@param {type:"number"}
seed_fix = False #@param {'type':'boolean'} 
#ループ処理
for i in range(N):
   if seed_fix == True:
    seed = seed
    generator = torch.Generator("cuda").manual_seed(seed)
   else:
    seed = random.randrange(0, 2147483647, 1)
    generator = torch.Generator("cuda").manual_seed(seed)
   with autocast("cuda"):
    pipe.enable_attention_slicing()
    pipe.enable_xformers_memory_efficient_attention()
    with torch.inference_mode():
    #images = pipe(prompt, height = height, width = width, guidance_scale = guidance_scale, num_inference_steps = num_inference_steps,negative_prompt = negative_prompt, generator = generator).images
     images = pipe.text2img(prompt, negative_prompt = negative_prompt, width = width,height = height,guidance_scale = guidance_scale, num_inference_steps = num_inference_steps,max_embeddings_multiples=8,generator = generator).images[0]

#どのSEED値で描画されたか確認用
     print("SEED =",seed)
     display(images)#.save(f'output{i}.png')

In [ ]:
#@title Gdriveからモデルデータを読込できるようにしたい
from google.colab import drive
drive.mount('/content/drive')

!python -m venv .env

!source .env/bin/activate

!pip install diffusers==0.7.1 transformers scipy ftfy accelerate
!pip install --upgrade diffusers transformers scipy
#!huggingface-cli login
import torch
from torch import autocast
# secretAI
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler

euler_ancestral_scheduler = EulerAncestralDiscreteScheduler.from_config("content/drive/MyDrive/model/novelAI.ckpt", subfolder="scheduler")
pipeline = StableDiffusionPipeline.from_pretrained(
    "content/drive/MyDrive/model/novelAI.ckpt",torch_dtype=torch.float16, custom_pipeline="lpw_stable_diffusion", scheduler=euler_scheduler, use_auth_token=YOUR_TOKEN,
)
pipeline.to("cuda")